# NNLS Permutation Pipeline

This notebook implements full NNLS + permutation-testing pipeline:

1. **Observed s₁**: highest beta per parcel per subject, averaged across subjects.
2. **Null distribution**: circularly shift each 160×160 trait RDM by a random TR, rerun NNLS, extract s₁, average → repeat `n_perm` times.
3. **Permutation p-value**: compare observed s₁ to null distribution for each parcel.
4. **Save** results to a dedicated output folder under `/Volumes/Passport/fmriprep/derivatives/RSA_stats/pieman`.


## 1. Imports

In [1]:
import os
import numpy as np
import pandas as pd
import random
from scipy.optimize import nnls
import os, glob, re, csv
import glob
import nibabel as nib
import statsmodels.api as sm
from nilearn import datasets
from nilearn.image import resample_to_img
import pandas as pd
from scipy.stats import ttest_1samp
from nilearn import plotting, datasets
import scipy.stats as stats
from statsmodels.stats.multitest import fdrcorrection
from tqdm import tqdm

## 2. Load Cleaned BOLD Data

In [2]:
# Load the BOLD cleaned image
bold_img = nib.load('/Volumes/Passport/fmriprep/derivatives/pieMan_cleaned/sub-002/func/sub-002_task-pieman_run-1_cleaned_desc-masked_bold.nii.gz')

# Print voxel size (spatial resolution) and TR (temporal resolution)
zooms = bold_img.header.get_zooms()
print(f"Voxel size (mm): {zooms[:3]}")
print(f"TR (s): {zooms[3]}")
print(f"Shape {bold_img.shape}")

Voxel size (mm): (3.0, 3.0, 4.0)
TR (s): 1.5
Shape (65, 77, 49, 160)


# 3. Set Parameters/Paths

In [ ]:
# SET MAIN HYPERPARAMETERS
# TRAIT_LABEL = "Contemplating"  

TRAIT_SETS = {
    "all_13": [
        "Open-minded","feeling Affectionate","Attentive","Assertive",
        "feeling Gloomy","feeling Peaceful","Agreeable","Judging",
        "feeling Angry","feeling Bewildered","Impulsive",
        "Self-disciplined","Contemplating"
    ],
    "mental_8": [
        "feeling Affectionate","feeling Gloomy","feeling Peaceful",
        "feeling Angry","feeling Bewildered","Judging",
        "Contemplating","Attentive"
    ],
    "personality_5": [
        "Open-minded","Agreeable","Assertive",
        "Self-disciplined","Impulsive"
    ],
    "trait_9": [
        "Open-minded","feeling Affectionate","Attentive","Assertive",
        "Agreeable","Judging","feeling Angry","Self-disciplined","Contemplating"
    ]
}

# Select model here: choose one key from TRAIT_SETS
model_key = "all_13"   # options: all_13, mental_8, personality_5, drop4_9
traits    = TRAIT_SETS[model_key]

# ALL_TRAIT_LABELS = [
    #"Open-minded","feeling Affectionate","Attentive","Assertive",
    #"feeling Gloomy","feeling Peaceful","Agreeable","Judging",
    #"feeling Angry","feeling Bewildered","Impulsive",
    #"Self-disciplined","Contemplating"
#]
ALL_TRAIT_SAVE_STRS = [t.replace(" ","_").replace("-","_")
                       for t in traits]
# Our 13 trait labels 
# ["Open-minded", "feeling Affectionate", "Attentive", "Assertive", "feeling Gloomy", "feeling Peaceful", "Agreeable", "Judging", "feeling Angry", "feeling Bewildered", "Impulsive", "Self-disciplined", "Contemplating"]

#TRAIT_LABEL_SAVE_STRING = TRAIT_LABEL.replace(" ", "_").replace("-", "_")
STIMULUS_LABEL_SAVE_STRING = "pieman"

# Set window shift amount
shift_window = 20  #shifts the window by x TRs 

# Set smoothing setting to either use smoothed trait RDMs or the un-smoothed RDMs
smoothing_setting = ""    # set to _no_smoothing or set to "" for smoothed RDMs

In [ ]:
# ──────────────────────────────────────────────────────────────
# 0) PATHS & I/O
# ──────────────────────────────────────────────────────────────
root_dir  = "/Volumes/Passport/fmriprep"          # ←  same as in cleaning script
deriv_dir = os.path.join(root_dir, "derivatives") #   (don’t hard-code “subjects” yet)



# output from your behaviour-model RSA
for trait_long, trait_save in zip(traits, ALL_TRAIT_SAVE_STRS):
    rdm_path = os.path.join(
        deriv_dir, "RDMs_behavior",
        f"{STIMULUS_LABEL_SAVE_STRING}_{trait_save}_RDM{smoothing_setting}.npy"
    )
    model_rdm = np.load(rdm_path)


perm_dir = os.path.join(
    deriv_dir, "RSA_stats", STIMULUS_LABEL_SAVE_STRING, "perm_test_trait_shift"
)

# 4. Filter Subjects

In [6]:
# ──────────────────────────────────────────────────────────────
# 1) SUBJECT / RUN FILTERS  (copy-paste verbatim)  ─────────────
# ──────────────────────────────────────────────────────────────
exclude_subs = {
    "sub-001","sub-021","sub-022","sub-038","sub-056","sub-068","sub-069"
}
exclude_sub_runs = {
    ("sub-002","2"),("sub-003","2"),("sub-004","2"),("sub-005","2"),("sub-006","2"),
    ("sub-008","2"),("sub-010","2"),("sub-011","2"),("sub-012","2"),("sub-013","2"),
    ("sub-014","2"),("sub-015","2"),("sub-016","2")
}
target_subject = None     # e.g. "sub-002" to run a single person


In [7]:
# ----------------------------------------------------------------
# 2)  BUILD SUBJECT LIST  (from cleaned derivatives)  ------------
# ----------------------------------------------------------------
cleaned_root = os.path.join(deriv_dir, f"{STIMULUS_LABEL_SAVE_STRING}_cleaned")
all_subs     = sorted(
    d for d in os.listdir(cleaned_root) if d.startswith("sub-")
)
if target_subject:
    if target_subject not in all_subs:
        raise ValueError(f"{target_subject} not found in {cleaned_root}")
    subjects = [target_subject]
else:
    subjects = [s for s in all_subs if s not in exclude_subs]

print("Subjects to process →", ", ".join(subjects))

Subjects to process → sub-002, sub-003, sub-004, sub-005, sub-006, sub-007, sub-008, sub-009, sub-010, sub-011, sub-012, sub-013, sub-014, sub-015, sub-016, sub-017, sub-018, sub-019, sub-020, sub-023, sub-024, sub-025, sub-026, sub-027, sub-028, sub-029, sub-030, sub-031, sub-032, sub-033, sub-034, sub-035, sub-036, sub-037, sub-039, sub-040, sub-041, sub-042, sub-043, sub-044, sub-045, sub-046, sub-047, sub-048, sub-049, sub-050, sub-051, sub-052, sub-053, sub-054, sub-055, sub-057, sub-058, sub-059, sub-060, sub-061, sub-062, sub-063, sub-064, sub-065, sub-066, sub-067, sub-070, sub-071, sub-072, sub-073, sub-074, sub-075, sub-076, sub-077, sub-078, sub-079, sub-080, sub-081, sub-082


# 5. Fetch Schaefer Atlas

In [8]:
# ──────────────────────────────────────────────────────────────
# FETCH SCHAEFER ATLAS  ─────────────────────────────────────
# ──────────────────────────────────────────────────────────────

# Schaefer parcel/atlas parameters
n_rois = 200
yeo_networks = 17
resolution_mm = 2                   # resolution of your Schaefer atlas (double check!)

schaefer    = datasets.fetch_atlas_schaefer_2018(
                 n_rois=n_rois,
                 yeo_networks=yeo_networks,
                 resolution_mm=resolution_mm
             )
atlas_img   = nib.load(schaefer['maps'])  # default 2mm MNI - but our images 3x3x4 (Pieman and others) OR 2.5^3 (ie., Black and Forgot)

atlas_resampled = resample_to_img(atlas_img, bold_img, interpolation='nearest')
atlas_data     = atlas_resampled.get_fdata()



# Change Schaeffer Labels so 0 is whole brain and 1 corresponds to 1st ROI
labels = schaefer['labels']
# change to string and remove excess
labels = [l.replace(b'17Networks_', b'').decode('utf-8') for l in labels]
# Prepend background label
labels = np.insert(labels, 0, "Background")

In [ ]:
# ─── PERMUTATION TESTING Pipeline (s₁…sₙ) ────────────────────────────────
from scipy.optimize import nnls
from tqdm import tqdm

def vectorize_rdm(rdm):
    idx = np.tril_indices(rdm.shape[0], k=-1)
    return rdm[idx]

# 9.1) load all subject‐run CSVs with betas 
multi_csvs = glob.glob(os.path.join(
    deriv_dir, "RSA_stats", STIMULUS_LABEL_SAVE_STRING,
    "multi_regression","subject_results",
    f"*_{STIMULUS_LABEL_SAVE_STRING}_multi_parcel_RSA_NNLS_{model_key}{smoothing_setting}.csv"
))

# assert we have one file per subject (add dictionary for different stimuli later)
assert len(multi_csvs) == 75, (                     
    f"Expected 75 files, but found {len(multi_csvs)}"
)

df_list = []
for fn in multi_csvs:
    df = pd.read_csv(fn)
    df['unit'] = df['subject'] + "_" + df['run'].astype(str)
    df_list.append(df)
all_df = pd.concat(df_list, ignore_index=True)

trait_cols = ALL_TRAIT_SAVE_STRS
n_traits  = len(trait_cols)

# 9.2) compute observed s_k per parcel for k=1…n_traits
observed = {}
# sort-and-sum top-k per row, then average by parcel
for k in range(1, n_traits+1):
    # descending sort, sum top k
    all_df[f's{k}'] = (
        np.sort(all_df[trait_cols].values, axis=1) # 1) sort each row ascending
        [:, ::-1]   # 2) reverse to descending 
        [:, :k]      # 3) take top-k
        .sum(axis=1)  # 4) sum across rows
    )
    observed[k] = all_df.groupby('parcel_num')[f's{k}'].mean()

# 9.3) precompute & cache every neural RDM (unchanged)
neural_rdm_cache = {}
for sub in subjects:
    func_dir = os.path.join(cleaned_root, sub, "func")
    run_pat  = os.path.join(func_dir,
        f"{sub}_task-{STIMULUS_LABEL_SAVE_STRING}_run-*_*cleaned_desc-masked_bold.nii.gz"
    )
    single_pat = os.path.join(func_dir,
        f"{sub}_task-{STIMULUS_LABEL_SAVE_STRING}_cleaned_desc-masked_bold.nii.gz"
    )
    bold_files = sorted(glob.glob(run_pat)) + sorted(glob.glob(single_pat))
    for bf in bold_files:
        m = re.search(r"_run-(\d+)_", os.path.basename(bf))
        run = m.group(1) if m else "NA"
        if (sub, run) in exclude_sub_runs: 
            continue
        print("Attempting to load:", bf)  # debug line for when files get corrupted
        img = nib.load(bf)
        atlas_res = resample_to_img(atlas_img, img, interpolation='nearest')
        atlas_dat = atlas_res.get_fdata().astype(int)
        bold_dat  = img.get_fdata()
        for pid in range(1, n_rois+1):
            mask = atlas_dat == pid
            if not mask.any(): 
                continue
            # 1 - corr → RDM
            corr = np.corrcoef(bold_dat[mask,:].T)
            neural_rdm_cache[(sub, run, pid)] = (1 - corr).astype(np.float32)

    

# 9.4) load behavior RDMs (unchanged)
behavior_rdms = {
    trait: np.load(os.path.join(
        deriv_dir, "RDMs_behavior",
        f"{STIMULUS_LABEL_SAVE_STRING}_{sstr}_RDM{smoothing_setting}.npy"
    ))
    for trait, sstr in zip(traits, ALL_TRAIT_SAVE_STRS)
}

# 9.5) build null distributions for sₖ
n_perm = 100
parcel_ids = list(observed[1].index)
nulls = {k: {pid: [] for pid in parcel_ids} for k in range(1, n_traits+1)}
n_tr   = next(iter(behavior_rdms.values())).shape[0]

assert n_tr == 160, f"Expected 160 timepoints, got {n_tr}"

for i in tqdm(range(n_perm), desc="Permutations"):
    # random circular TR shift ≥20
    shift = np.random.randint(shift_window, n_tr-shift_window)
    permuted = {
        trait: vectorize_rdm(np.roll(np.roll(mat, shift, axis=0),
                                     shift, axis=1))
        for trait, mat in behavior_rdms.items()
    }

    # accumulate sums & counts for each k & parcel
    sums   = {k: {pid: 0.0 for pid in parcel_ids} for k in range(1, n_traits+1)}
    counts = {k: {pid: 0   for pid in parcel_ids} for k in range(1, n_traits+1)}

    for (sub, run, pid), nr in neural_rdm_cache.items():
        y = vectorize_rdm(nr)
        X = np.column_stack([np.ones_like(y)] +
                             [permuted[t] for t in traits])
        coef, _    = nnls(X, y)
        betas      = coef[1:]                     # drop intercept
        top_sorted = np.sort(betas)[::-1]         # descending

        # for each k, sum top-k betas
        for k in range(1, n_traits+1):
            s_k = top_sorted[:k].sum()
            sums[k][pid]   += s_k
            counts[k][pid] += 1

    # record mean null s_k for each parcel
    for k in range(1, n_traits+1):
        for pid in parcel_ids:
            nulls[k][pid].append(sums[k][pid] / counts[k][pid])

# 9.6) compare observed vs null → p-values for every (k, parcel)
results = []
for k in range(1, n_traits+1):
    for pid, obs_val in observed[k].items():
        dist = np.array(nulls[k][pid])
        pval = (np.sum(dist >= obs_val) + 1) / (n_perm + 1)
        results.append((k, pid, obs_val, pval, pval < .05))

df_perm = pd.DataFrame(
    results,
    columns=['k', 'parcel_num', 'observed_s', 'p_value', 'significant_p05']
)

# 9.7) add parcel labels and save
parcel_labels = pd.DataFrame({
    'parcel_num': np.arange(1, len(labels)),
    'parcel_label': labels[1:]
})
# ─── define & create output folder ─────────────────────────────────────────
perm_dir = os.path.join(
    deriv_dir, "RSA_stats", STIMULUS_LABEL_SAVE_STRING, "perm_test_trait_shift"
)
os.makedirs(perm_dir, exist_ok=True)

df_perm = df_perm.merge(parcel_labels, on='parcel_num')
out_csv = os.path.join(perm_dir, f"perm_test_trait_shift_{model_key}_allks{smoothing_setting}.csv")
df_perm.to_csv(out_csv, index=False)
print(f"✅ Permutation results for k=1…{n_traits} saved → {out_csv}")

Attempting to load: /Volumes/Passport/fmriprep/derivatives/pieman_cleaned/sub-002/func/sub-002_task-pieman_run-1_cleaned_desc-masked_bold.nii.gz
Attempting to load: /Volumes/Passport/fmriprep/derivatives/pieman_cleaned/sub-003/func/sub-003_task-pieman_run-1_cleaned_desc-masked_bold.nii.gz
Attempting to load: /Volumes/Passport/fmriprep/derivatives/pieman_cleaned/sub-004/func/sub-004_task-pieman_run-1_cleaned_desc-masked_bold.nii.gz
Attempting to load: /Volumes/Passport/fmriprep/derivatives/pieman_cleaned/sub-005/func/sub-005_task-pieman_run-1_cleaned_desc-masked_bold.nii.gz
Attempting to load: /Volumes/Passport/fmriprep/derivatives/pieman_cleaned/sub-006/func/sub-006_task-pieman_run-1_cleaned_desc-masked_bold.nii.gz
Attempting to load: /Volumes/Passport/fmriprep/derivatives/pieman_cleaned/sub-007/func/sub-007_task-pieman_cleaned_desc-masked_bold.nii.gz
Attempting to load: /Volumes/Passport/fmriprep/derivatives/pieman_cleaned/sub-008/func/sub-008_task-pieman_run-1_cleaned_desc-masked_bol

Permutations: 100%|██████████| 100/100 [17:03<00:00, 10.23s/it]

✅ Permutation results for k=1…5 saved → /Volumes/Passport/fmriprep/derivatives/RSA_stats/pieman/permutation_testing/perm_test_personality_5_allks.csv


In [64]:
observed_df = pd.DataFrame(observed)
print(observed_df)

                   1         2         3         4         5
parcel_num                                                  
1           0.002266  0.002461  0.002461  0.002461  0.002461
2           0.002344  0.002636  0.002636  0.002636  0.002636
3           0.002187  0.002386  0.002386  0.002386  0.002386
4           0.002188  0.002407  0.002407  0.002407  0.002407
5           0.002144  0.002294  0.002294  0.002294  0.002294
...              ...       ...       ...       ...       ...
196         0.001971  0.002019  0.002019  0.002019  0.002019
197         0.002024  0.002112  0.002112  0.002112  0.002112
198         0.002097  0.002145  0.002145  0.002145  0.002145
199         0.002123  0.002211  0.002211  0.002211  0.002211
200         0.002030  0.002143  0.002143  0.002143  0.002143

[200 rows x 5 columns]


In [65]:
df_perm = pd.read_csv(out_csv)
print(df_perm.info())
print(df_perm.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   k                1000 non-null   int64  
 1   parcel_num       1000 non-null   int64  
 2   observed_s       1000 non-null   float64
 3   p_value          1000 non-null   float64
 4   significant_p05  1000 non-null   bool   
 5   parcel_label     1000 non-null   object 
dtypes: bool(1), float64(2), int64(2), object(1)
memory usage: 40.2+ KB
None
   k  parcel_num  observed_s  p_value  significant_p05          parcel_label
0  1           1    0.002266      1.0            False    LH_VisCent_ExStr_1
1  1           2    0.002344      1.0            False    LH_VisCent_ExStr_2
2  1           3    0.002187      1.0            False  LH_VisCent_Striate_1
3  1           4    0.002188      1.0            False    LH_VisCent_ExStr_3
4  1           5    0.002144      1.0            False    LH_VisC

## Create Whole Brain Parcellation Map of p-values

In [ ]:
# Load CSV file 
perm_csv_path = f"/Volumes/Passport/fmriprep/derivatives/RSA_stats/pieman/permutation_testing/perm_test_trait_shift{model_key}_allks.csv"
df_perm1 = pd.read_csv(perm_csv_path)

# Filter to only the k=1 results
df_perm1['k'] = df_perm1['k'].astype(int)
df_k1 = df_perm1[df_perm1['k'] == 1].copy()
df_k1['parcel_num'] = df_k1['parcel_num'].astype(int)


# 1) Extract the integer parcel map
atlas_data = atlas_resampled.get_fdata().astype(int)

# 2) Create maps
p_map               = np.zeros(atlas_data.shape, dtype=float)               # unthresholded
p_map_thresholded   = np.full(atlas_data.shape, np.nan, dtype=float)        # use NaN for masking
s_map               = np.zeros(atlas_data.shape, dtype=float)               # unthresholded
s_map_thresholded   = np.zeros(atlas_data.shape, dtype=float)               # 0 for non-sig

# 3) Fill in each parcel
for _, row in df_k1.iterrows():
    pid  = int(row['parcel_num'])
    pval = float(row['p_value'])
    sval = float(row['observed_s'])

    p_map[atlas_data == pid] = pval
    s_map[atlas_data == pid] = sval

    if pval < 0.05:
        p_map_thresholded[atlas_data == pid] = pval
        s_map_thresholded[atlas_data == pid] = sval

# 4) Wrap and save NIfTI images
p_img = nib.Nifti1Image(p_map, atlas_resampled.affine, atlas_resampled.header)
nib.save(p_img, os.path.join(perm_dir, f"perm_pmap_trait_shift_{model_key}_k1.nii.gz"))
print("✅ unthresholded k=1 p-value map saved")

p_img_thresh = nib.Nifti1Image(p_map_thresholded, atlas_resampled.affine, atlas_resampled.header)
nib.save(p_img_thresh, os.path.join(perm_dir, f"perm_pmap_trait_shift_{model_key}_k1_thresh05.nii.gz"))
print("✅ thresholded k=1 p-value map (NaN-masked) saved")

s_img = nib.Nifti1Image(s_map, atlas_resampled.affine, atlas_resampled.header)
nib.save(s_img, os.path.join(perm_dir, f"perm_smap_trait_shift_{model_key}_k1.nii.gz"))
print("✅ k=1 observed s-value map saved")

s_img_thresh = nib.Nifti1Image(s_map_thresholded, atlas_resampled.affine, atlas_resampled.header)
nib.save(s_img_thresh, os.path.join(perm_dir, f"perm_smap_trait_shift_{model_key}_k1_thresh05.nii.gz"))
print("✅ thresholded k=1 s-value map saved")

# Optional: Summary for verification
n_sig_parcels = df_k1[df_k1['p_value'] < 0.05]['parcel_num'].nunique()
print(f"Significant parcels (p < 0.05): {n_sig_parcels}")
print("Nonzero thresholded voxels in s_map:", np.count_nonzero(s_map_thresholded))
print("Non-NaN voxels in thresholded p_map:", np.count_nonzero(~np.isnan(p_map_thresholded)))



✅ unthresholded k=1 p-value map saved
✅ thresholded k=1 p-value map (NaN-masked) saved
✅ k=1 observed s-value map saved
✅ thresholded k=1 s-value map saved
Significant parcels (p < 0.05): 3
Nonzero thresholded voxels in s_map: 467
Non-NaN voxels in thresholded p_map: 467
